In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_score

# Mais imports
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import precision_score

In [2]:
# Análise da base
df = pd.read_csv('games.csv')
df.describe()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
count,5.149000e+04,5.149000e+04,51490.000000,51490.0,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,...,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000
mean,3.306223e+09,1.502926e+12,1832.362808,9.0,1.493552,1.471295,1.450631,1.308487,0.926510,1.442804,...,5.549466,0.985084,0.414547,1.404370,0.240105,108.216294,107.910216,108.690581,108.626044,108.066576
std,2.946096e+07,1.978026e+09,512.017696,0.0,0.499963,0.520326,0.542848,0.676097,0.841424,0.569579,...,3.860989,1.256284,0.613768,1.224492,0.427151,102.551787,102.870710,102.592145,103.346952,102.756149
min,3.214824e+09,1.496892e+12,190.000000,9.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,3.292218e+09,1.502021e+12,1531.000000,9.0,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,38.000000,37.000000,38.000000,38.000000,38.000000
50%,3.320021e+09,1.503844e+12,1833.000000,9.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,6.000000,0.000000,0.000000,1.000000,0.000000,90.000000,90.000000,90.000000,90.000000,90.000000
75%,3.327099e+09,1.504352e+12,2148.000000,9.0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,9.000000,2.000000,1.000000,2.000000,0.000000,141.000000,141.000000,141.000000,141.000000,141.000000
max,3.331833e+09,1.504707e+12,4728.000000,9.0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,11.000000,10.000000,4.000000,6.000000,1.000000,516.000000,516.000000,516.000000,516.000000,516.000000


In [3]:
df['t1_champ3_sum2']


0         7
1         3
2         4
3         7
4         3
         ..
51485    14
51486     4
51487     4
51488    12
51489    12
Name: t1_champ3_sum2, Length: 51490, dtype: int64

1 - Classificar entre vencedor baseado objetivos feitos.

In [4]:
# Separando dados
X_t1 = df[['t1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 'gameDuration']].copy()
X_t1

X_t2 = df[['t2_towerKills', 't2_inhibitorKills', 't2_baronKills',
       't2_dragonKills', 't2_riftHeraldKills', 'gameDuration']].copy()
X_t2
y = df[['winner']] - 1

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_t1, y, test_size = 0.2)
print(X_train.shape)
print(X_test.shape)

(41192, 6)
(10298, 6)


In [6]:
# Z-Normaliza os dados
mean_train = X_train.mean()
std_train = X_train.std(ddof=1)

Z_train = X_train.copy()
Z_train = X_train - mean_train
Z_train = Z_train/std_train
Z_train

,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,gameDuration
37711,0.606592,-0.013809,1.070593,0.507896,-0.582501,-0.565350
38911,1.396728,3.150658,1.070593,1.336853,-0.582501,4.260208
21463,-1.500437,-0.804925,-0.637512,-1.150020,-0.582501,-3.175865
9313,0.869971,1.568425,1.070593,0.507896,-0.582501,0.858745
42522,-0.710301,-0.804925,-0.637512,-1.150020,-0.582501,-0.107535
...,...,...,...,...,...,...
8776,-0.183544,-0.013809,1.070593,-0.321062,-0.582501,0.907449
24439,0.079835,-0.013809,-0.637512,-1.150020,1.716693,-0.522490
26814,0.606592,0.777308,-0.637512,0.507896,-0.582501,0.069746
34336,-1.237059,-0.804925,-0.637512,-1.150020,-0.582501,-0.563401


In [7]:
skf = StratifiedKFold(n_splits=5, shuffle = True)
fold = 1
for train_index, val_index in skf.split(Z_train, y_train):
    print('{}º fold'.format(fold))
    for k in [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]:
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(Z_train.values[train_index], y_train.values[train_index])

        y_val = y_train.values[val_index]
        y_hat = model.predict(Z_train.values[val_index])

        precision = precision_score(y_val, y_hat)
        print('{} cluster(s) com {}% de precisão'.format(k, round(precision,3)*100))
    print('')
    print('')
    fold += 1

1º fold
1 cluster(s) com 91.60000000000001% de precisão
3 cluster(s) com 93.0% de precisão
5 cluster(s) com 93.4% de precisão
7 cluster(s) com 93.60000000000001% de precisão
9 cluster(s) com 94.0% de precisão
11 cluster(s) com 94.0% de precisão
13 cluster(s) com 94.1% de precisão
15 cluster(s) com 94.19999999999999% de precisão
17 cluster(s) com 94.6% de precisão
19 cluster(s) com 94.6% de precisão
21 cluster(s) com 94.8% de precisão


2º fold
1 cluster(s) com 92.4% de precisão
3 cluster(s) com 94.19999999999999% de precisão
5 cluster(s) com 94.3% de precisão
7 cluster(s) com 94.3% de precisão
9 cluster(s) com 94.69999999999999% de precisão
11 cluster(s) com 94.69999999999999% de precisão
13 cluster(s) com 95.0% de precisão
15 cluster(s) com 95.0% de precisão
17 cluster(s) com 94.6% de precisão
19 cluster(s) com 94.5% de precisão
21 cluster(s) com 94.8% de precisão


3º fold
1 cluster(s) com 92.0% de precisão
3 cluster(s) com 93.0% de precisão
5 cluster(s) com 94.1% de precisão
7 clust

2 - Classificar entre vencedor ou não baseado nos bans dos campeões.

In [8]:
# Separando dados
X = df[['t2_ban1', 't2_ban2', 't2_ban3', 't2_ban4',
       't2_ban5']].copy()
X
y = df[['winner']] - 1

In [9]:
# Separa em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape)
print(X_test.shape)

(41192, 5)
(10298, 5)


In [10]:
skf = StratifiedKFold(n_splits=5, shuffle = True)
fold = 1
for train_index, val_index in skf.split(X_train, y_train):
    print('{}º fold'.format(fold))
    for k in [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]:
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train.values[train_index], y_train.values[train_index])

        y_val = y_train.values[val_index]
        y_hat = model.predict(X_train.values[val_index])

        precision = precision_score(y_val, y_hat)
        print('{} cluster(s) com {}% de precisão'.format(k, round(precision,3)*100))
    print('')
    print('')
    fold += 1

1º fold
1 cluster(s) com 49.2% de precisão
3 cluster(s) com 48.6% de precisão
5 cluster(s) com 48.4% de precisão
7 cluster(s) com 48.9% de precisão
9 cluster(s) com 48.9% de precisão
11 cluster(s) com 48.8% de precisão
13 cluster(s) com 48.699999999999996% de precisão
15 cluster(s) com 49.6% de precisão
17 cluster(s) com 49.3% de precisão
19 cluster(s) com 49.4% de precisão
21 cluster(s) com 49.2% de precisão


2º fold
1 cluster(s) com 49.7% de precisão
3 cluster(s) com 49.1% de precisão
5 cluster(s) com 49.3% de precisão
7 cluster(s) com 49.5% de precisão
9 cluster(s) com 49.5% de precisão
11 cluster(s) com 49.9% de precisão
13 cluster(s) com 49.5% de precisão
15 cluster(s) com 49.2% de precisão
17 cluster(s) com 48.9% de precisão
19 cluster(s) com 49.2% de precisão
21 cluster(s) com 49.0% de precisão


3º fold
1 cluster(s) com 50.3% de precisão
3 cluster(s) com 48.699999999999996% de precisão
5 cluster(s) com 48.9% de precisão
7 cluster(s) com 50.0% de precisão
9 cluster(s) com 49.9%